# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points("Lng",
                                            "Lat",
                                            frame_width=800,
                                            frame_height=600,
                                            hover_cols="all",
                                            size="Humidity",
                                            alpha=0.5,
                                            geo=True,
                                            tiles=True)

# Display the map
city_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,index,City_ID,City,Max Temp,Cloudiness,Wind Speed,Country,Date)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[ (city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 30) \
                              & (city_data_df['Humidity'] < 60) \
                              & (city_data_df['Cloudiness'] == 0) ].copy()
# Drop any rows with null values
ideal_city_df.dropna(axis=0, inplace=True)

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436
363,363,narwar,25.6500,77.9000,22.35,55,0,1.29,IN,1666108449
391,391,saint-francois,46.4154,3.9054,23.69,57,0,4.12,FR,1666108465
436,436,lakki marwat,32.6079,70.9114,27.89,20,0,1.53,PK,1666108492
496,496,tikrit,34.6071,43.6782,29.02,14,0,1.41,IQ,1666108524
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lng","Lat","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
63,banda,IN,80.3333,25.4833,52,
73,nalut,LY,10.8500,30.3333,27,
240,tikaitnagar,IN,81.5833,26.9500,59,
258,comodoro rivadavia,AR,-67.5000,-45.8667,24,
340,santa rosalia,MX,-112.2833,27.3167,56,
363,narwar,IN,77.9000,25.6500,55,
391,saint-francois,FR,3.9054,46.4154,57,
436,lakki marwat,PK,70.9114,32.6079,20,
496,tikrit,IQ,43.6782,34.6071,14,
550,pali,IN,73.3333,25.7667,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
lakki marwat - nearest hotel: No hotel found
tikrit - nearest hotel: فندق بلازا
pali - nearest hotel: Hotel Pawan International


,City,Country,Lng,Lat,Humidity,Hotel Name
63,banda,IN,80.3333,25.4833,52,#acnindiafy21
73,nalut,LY,10.8500,30.3333,27,No hotel found
240,tikaitnagar,IN,81.5833,26.9500,59,No hotel found
258,comodoro rivadavia,AR,-67.5000,-45.8667,24,No hotel found
340,santa rosalia,MX,-112.2833,27.3167,56,Sol y Mar
363,narwar,IN,77.9000,25.6500,55,No hotel found
391,saint-francois,FR,3.9054,46.4154,57,Chez Lily
436,lakki marwat,PK,70.9114,32.6079,20,No hotel found
496,tikrit,IQ,43.6782,34.6071,14,فندق بلازا
550,pali,IN,73.3333,25.7667,26,Hotel Pawan International


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols='all',#['Lng','Lat','City','Humidity','Hotel Name','Country']
    geo=True,
    tiles=True,#"EsriImagery",
    frame_width=800,
    frame_height=600,
    size="Humidity")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,index,City,Country,Hotel Name)